##  Cynaptics Bonus Induction Task

in this task  i used  ngram and tranformer approach earlier i used lstm its hangmanguessmodel is attached at bottom there was a 8 % difference between the accuracy and transformer performed  better 


In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
class HangmanGuessModel(nn.Module):
    def __init__(self, input_size_word=32, input_size_letters=26, hidden_size=256, output_size=26, num_layers=3, dropout_rate=0.3, ngram_range=(2, 2), max_seq_len=30):
        super(HangmanGuessModel, self).__init__()
        assert input_size_word % 4 == 0, "input_size_word must be divisible by 4 for multi-head attention."
        self.embedding = nn.Embedding(28, input_size_word)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=input_size_word, nhead=4, dropout=dropout_rate)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=num_layers)
        self.guessed_fc = nn.Linear(input_size_letters, hidden_size)
        self.combine_fc = nn.Linear(hidden_size + input_size_word, hidden_size)
        self.output_fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.vectorizer = CountVectorizer(ngram_range=ngram_range, analyzer='char')
        self.positional_encoding = nn.Parameter(torch.zeros(1, max_seq_len, input_size_word))

    def preprocess_word(self, word):
        """
        Preprocess the word by converting letters to numerical indices, treating "_" as 27.
        Also, extract n-grams (e.g., bi-grams) and convert them to features.
        """
        clean_word = word.replace(" ", "") 
        input_tensor = torch.zeros((1, len(clean_word)), dtype=torch.long) 
        ngrams = self.vectorizer.fit_transform([clean_word]).toarray()
        for pos, letter in enumerate(clean_word):
            if letter == '_':
                input_tensor[0, pos] = 27  
            else:
                input_tensor[0, pos] = ord(letter) - ord('a')  

        return input_tensor, ngrams
    def create_guess_tensors(self, guessed_letters):
        """
        Create a tensor representing guessed letters (binary vector for letters a-z).
        """
        guessed_tensor = torch.zeros(1, 26)  
        for letter in guessed_letters:
            index = ord(letter) - ord('a')
            if 0 <= index < 26:  
                guessed_tensor[0][index] = 1
        return guessed_tensor

    def forward(self, masked_word, guessed_letters):
        """
        Forward pass for the HangmanTransformerModel.
        """
        masked_word_tensor, ngram_features = self.preprocess_word(masked_word)
        guessed_letters_tensor = self.create_guess_tensors(guessed_letters)
        embedded_word = self.embedding(masked_word_tensor)  
        embedded_word = embedded_word + self.positional_encoding[:, :embedded_word.size(1), :]
        transformer_out = self.transformer_encoder(embedded_word.transpose(0, 1)) 
        transformer_out = transformer_out.transpose(0, 1)  
        final_transformer_output = transformer_out[:, -1, :]  
        guessed_output = self.guessed_fc(self.dropout(guessed_letters_tensor))  
        combined_input = torch.cat((final_transformer_output, guessed_output), dim=1)
        combined_output = self.combine_fc(self.dropout(combined_input))
        logits = self.output_fc(self.dropout(combined_output))

        return logits


Example Use

In [16]:
model = HangmanGuessModel()

model.eval()

logits = model('c y n a _ t i c s' , ['c', 'n', 'a', 't', 'i', 's', 'v', 'u', 'e' , 'y'])

logits

/opt/anaconda3/envs/dsenv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


tensor([[ 0.0193,  0.2625, -0.0622,  0.1439,  0.2566,  0.2268,  0.1836,  0.0985,
         -0.0446, -0.0339, -0.0806, -0.0230, -0.2673,  0.0872,  0.3334, -0.1363,
          0.0528,  0.0012,  0.1029,  0.1034,  0.0091, -0.0099,  0.1059, -0.0880,
          0.0401, -0.1034]], grad_fn=<AddmmBackward0>)

Hangman Game

In [17]:
import random
import string
import torch

class CynapticsHangman():

  def __init__(self):
        self.guessed_letters = []     # your current guessed letters
        self.train_dataset_location = "train.txt" # train dataset location
        self.validation_dataset_location = "valid.txt" # validation dataset location

        self.train_dictionary = self.build_dictionary(self.train_dataset_location)
        self.validation_dictionary = self.build_dictionary(self.validation_dataset_location)
        self.lives_remaining = 6 # number of lives remaining in the game
        self.model = HangmanGuessModel()
        
  # to load your model for evaluation
  def init_model(self , model_path ):
        self.model = self.load_model_weights(model_path)

  # a function to build a dictionary from a dataset
  def build_dictionary(self , dictionary_file_location):
    text_file = open(dictionary_file_location,"r")
    full_dictionary = text_file.read().splitlines()
    text_file.close()
    return full_dictionary

  # the guess function. Put your model here to predict the next guessed letter based on th einput masked word and already guessed letters
  # IT IS ADVISIBLE TO ONLY CHANGE THIS SECTION OF CODE
  def guess(self , masked_word , lives_left):
    # function that returns the guessed word
    # you are to put your model here
    # you can access the guessed words , masked word , lives remaining in this function to make your prediction

    # an example input maybe as follows
    # masked_word: 'c n a _ t i c s'   lives_remaining: 2    guessed_letters: ['c', 'n', 'a', 't', 'i', 's', 'v', 'u', 'e']    your next guess should be ideally 'p'.
    # Given below is a ML model based approach
    probabilities = self.model(masked_word , self.guessed_letters)
    mask = torch.ones_like(probabilities)  # Start with a mask of ones
    for letter in self.guessed_letters:
      if letter != ' ':
        letter_index = ord(letter) - 97  # Convert letter to index (a -> 0, b -> 1, ..., z -> 25)
        mask[0][letter_index] = 0  # Set the probability of already guessed letters to zero

    # Apply the mask to the probabilities
    masked_probabilities = probabilities * mask

    # Now select the letter with the highest probability that hasn't been guessed yet
    predicted_letter_index = torch.argmax(masked_probabilities).item()
    guessed_letter = chr(predicted_letter_index + 97)
    return guessed_letter


  # Save the model weights
  def save_model_weights(self , model, file_path):
      """
      Save model weights to a file.

      Args:
          model (nn.Module): The model whose weights are to be saved.
          file_path (str): Path to save the weights.
      """
      torch.save(model.state_dict(), file_path)
      print(f"Model weights saved to {file_path}")

  # Load the model weights
  def load_model_weights(self , file_path):
      """
      Load model weights from a file.

      Args:
          file_path (str): Path to the saved weights file.
      """
      model = HangmanGuessModel()
      model.load_state_dict(torch.load(file_path))
      model.eval()  # Set the model to evaluation mode
      print(f"Model weights loaded from {file_path}")
      return model


  def train(self , total_episodes = 10): # take as an input the total episodes the model trains for
      
    print("Training started")
    model = HangmanGuessModel()
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for episode in range(total_episodes):

        print(f"Episode {episode}")
        word = random.choice(self.train_dictionary)
        masked_word = '_' * len(word)

        guessed_letters = []
        lives_remaining = 6
        ground_truths = []
        predicted_truths = []
        max_tries = len(word) + 6  # Maximum number of tries is the length of the word + 6


        while lives_remaining > 0 and '_' in masked_word and max_tries > 0:
            max_tries -= 1
            # Format the masked word for the model
            input_to_model = ' '.join(masked_word)  # e.g., "_ _ _ _ _"

            probabilities = model(input_to_model , guessed_letters) # Get the probabilities of each letter

            # Create a tensor to mask out the probabilities of already guessed letters
            mask = torch.ones_like(probabilities)  # Start with a mask of ones
            for letter in guessed_letters:
                letter_index = ord(letter) - 97  # Convert letter to index (a -> 0, b -> 1, ..., z -> 25)
                mask[0][letter_index] = 0  # Set the probability of already guessed letters to zero

            # Apply the mask to the probabilities
            masked_probabilities = probabilities * mask

            # Now select the letter with the highest probability that hasn't been guessed yet
            predicted_letter_index = torch.argmax(masked_probabilities).item()
            guessed_letter = chr(predicted_letter_index + 97)


            # Prepare ground truth
            ground_truth = torch.zeros((1, 26)).to(device)  # Assuming 26 letters
            for letter in word:
                if letter not in masked_word:
                    ground_truth[0, ord(letter) - 97] = 1
                    
            # Update masked word and lives remaining
            if guessed_letter in word:
                masked_word = ''.join(
                    [c if c in guessed_letters + [guessed_letter] else '_' for c in word]
                )
            else:
                lives_remaining -= 1

            guessed_letters.append(guessed_letter)
            ground_truths.append(ground_truth)
            predicted_truths.append(probabilities)


        # Compute loss
        total_loss = 0
        for i in range(len(ground_truths)):
            total_loss += criterion(predicted_truths[i], ground_truths[i])

        # Backpropagation
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        if (episode + 1) % 1 == 0:
            print(f'Episode: {episode + 1}, Loss: {total_loss.item()}')
            
    # File path to save/load weights
    weights_file = "hangman_model_weights.pth"
    self.save_model_weights(model, weights_file)

  # function which returns the status of the current game based on the prediction
  def return_status (self , unmasked_word , masked_word  , guessed_letter):
      self.max_tries -= 1
      if guessed_letter in unmasked_word:
                masked_word = ''.join(
                    [c if c in self.guessed_letters else '_' for c in unmasked_word]
                )
                if '_' in masked_word:
                  return "ongoing" , self.lives_remaining , "Guess correct" , masked_word # successful guess
                else :
                  return "success" , self.lives_remaining , "Guess correct" , masked_word # successful guess and game over
      else:
                self.lives_remaining -= 1
                if self.lives_remaining == 0 :
                  return "failed" , self.lives_remaining , "Guess incorrect" , masked_word # unsuccessful guess
                else:
                  return "ongoing" , self.lives_remaining , "Guess incorrect" , masked_word # unsuccessful guess and game over


  # function to start a game
  def start_game(self, game_id , verbose=True):

        self.guessed_letters = [] #guessed letters
        self.guessed_letters.append(' ')
        word = random.choice(self.validation_dictionary) # a random word from the validation dataset
        masked_word = '_' * len(word)
        masked_word = ' '.join(masked_word)
        word = ' '.join(word)
        self.lives_remaining = 6 # default number of lives remianing at the start of the game

        if verbose:
              print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}. Masked Word: {3}.".format(game_id, self.lives_remaining, word , masked_word))

        self.max_tries = len(word) + 6 # maximum rounds a game can run for

        while self.lives_remaining > 0 and self.max_tries > 0 :

                guess_letter = self.guess(masked_word , self.lives_remaining)
                self.guessed_letters.append(guess_letter)

                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                status , self.lives_remaining , res , masked_word = self.return_status(word , masked_word , guess_letter)

                if verbose:
                    print("Sever response: {0}. Masked Word: {1}. Lives remaining: {2}".format(res , masked_word , self.lives_remaining))

                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True , 1

                elif status=="failed":
                    reason = '# of tries exceeded!'
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False , 0

Model Training



In [18]:
# creating an instance
hangman_game = CynapticsHangman()

episodes_to_train_for = 500  # number of episodes to train the model for
# training the model
hangman_game.train(episodes_to_train_for) 

# loading the model
hangman_game.init_model('hangman_model_weights.pth')



Training started
Episode 0
Episode: 1, Loss: 187.0795440673828
Episode 1
Episode: 2, Loss: 230.11732482910156
Episode 2
Episode: 3, Loss: 195.2206268310547
Episode 3
Episode: 4, Loss: 179.7694549560547
Episode 4
Episode: 5, Loss: 272.3489685058594
Episode 5
Episode: 6, Loss: 111.32657623291016
Episode 6
Episode: 7, Loss: 96.10591125488281
Episode 7
Episode: 8, Loss: 148.62924194335938
Episode 8
Episode: 9, Loss: 150.38661193847656
Episode 9
Episode: 10, Loss: 168.76475524902344
Episode 10
Episode: 11, Loss: 177.4821014404297
Episode 11
Episode: 12, Loss: 212.2898712158203
Episode 12
Episode: 13, Loss: 102.22477722167969
Episode 13
Episode: 14, Loss: 96.91813659667969
Episode 14
Episode: 15, Loss: 99.10589599609375
Episode 15
Episode: 16, Loss: 204.5537872314453
Episode 16
Episode: 17, Loss: 94.76156616210938
Episode 17
Episode: 18, Loss: 136.0032196044922
Episode 18
Episode: 19, Loss: 93.83676147460938
Episode 19
Episode: 20, Loss: 221.35574340820312
Episode 20
Episode: 21, Loss: 104.3

/var/folders/_b/zydhjjv91wg5ln53dtdp67sw0000gn/T/ipykernel_43799/482076456.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(file_path))


Model Evaluation

Try our your model for atleast 100 games

In [19]:
win = 0 # number of games won
played = 0 # number of games played
games_to_play = 100 # number of games to play

for i in range(games_to_play):
    try :
        print('Playing ', i+1 , ' th game')
        status , state = hangman_game.start_game(game_id = i , verbose=True)
        win += state
        played+=1
        print()
    except Exception as e:
        print("Error " , e)

try:
    success_rate = win/played
    print('Overall Success Rate = %.3f' % success_rate)
except Exception as e:
        print("Error " , e)
        print("Games Won: ", win)
        print("Games Played: ",played)

Playing  1  th game
Successfully start a new game! Game ID: 0. # of tries remaining: 6. Word: t e t r a o n i d. Masked Word: _ _ _ _ _ _ _ _ _.
Guessing letter: e
Sever response: Guess correct. Masked Word: _ e _ _ _ _ _ _ _. Lives remaining: 6
Guessing letter: a
Sever response: Guess correct. Masked Word: _ e _ _ a _ _ _ _. Lives remaining: 6
Guessing letter: r
Sever response: Guess correct. Masked Word: _ e _ r a _ _ _ _. Lives remaining: 6
Guessing letter: s
Sever response: Guess incorrect. Masked Word: _ e _ r a _ _ _ _. Lives remaining: 5
Guessing letter: n
Sever response: Guess correct. Masked Word: _ e _ r a _ n _ _. Lives remaining: 5
Guessing letter: d
Sever response: Guess correct. Masked Word: _ e _ r a _ n _ d. Lives remaining: 5
Guessing letter: t
Sever response: Guess correct. Masked Word: t e t r a _ n _ d. Lives remaining: 5
Guessing letter: i
Sever response: Guess correct. Masked Word: t e t r a _ n i d. Lives remaining: 5
Guessing letter: c
Sever response: Guess inco

In [ ]:
class HangmanGuessModel(nn.Module):
    def __init__(self, input_size_word=30, input_size_letters=26, hidden_size=256, output_size=26, num_layers=3, dropout_rate=0.3):
        super(HangmanGuessModel, self).__init__()
        self.embedding = nn.Embedding(28, input_size_word)
        self.lstm = nn.LSTM(input_size_word, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=dropout_rate)
        self.guessed_fc = nn.Linear(input_size_letters, hidden_size * 2) 
        self.combine_fc = nn.Linear(538, hidden_size) 
        self.output_fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)

    def preprocess_word(self, word):
        """
        Preprocess the word by converting letters to numerical indices, treating "_" as 27.
        """
        clean_word = word.replace(" ", "")  
        input_tensor = torch.zeros((1, len(clean_word)), dtype=torch.long) 

        for pos, letter in enumerate(clean_word):
            if letter == '_':
                input_tensor[0, pos] = 27  
            else:
                input_tensor[0, pos] = ord(letter) - ord('a')  
        return input_tensor

    def create_guess_tensors(self, guessed_letters):
        """
        Create a tensor representing guessed letters (binary vector for letters a-z).
        """
        guessed_tensor = torch.zeros(1, 26)  
        for letter in guessed_letters:
            index = ord(letter) - ord('a')
            if 0 <= index < 26:  
                guessed_tensor[0][index] = 1
        return guessed_tensor

    def forward(self, masked_word, guessed_letters):
        """
        Forward pass for the HangmanGuessModel.
        """
        masked_word_tensor = self.preprocess_word(masked_word)
        guessed_letters_tensor = self.create_guess_tensors(guessed_letters)
        embedded_word = self.embedding(masked_word_tensor)  
        lstm_out, _ = self.lstm(embedded_word)  
        final_lstm_output = lstm_out[:, -1, :] 
        combined_input = torch.cat((final_lstm_output, guessed_letters_tensor), dim=1)  
        combined_output = self.combine_fc(self.dropout(combined_input))
        logits = self.output_fc(self.dropout(combined_output))

        return logits
